In [1]:
import pandas as pd
from numpy import sin,cos,arctan2,sqrt,pi
import math
from tqdm import tqdm
from collections import Counter
import numpy as np
from sklearn.cluster import DBSCAN

import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
import urllib3
import requests
import json
import re

In [ ]:
yandex_token='911e9472-74b0-4bf7-8581-9f49fcc777fd'
rad = 6372795

def searchCoordinates(address):
    preprocLine = re.sub(',\s*','+', address)
    request = f'https://geocode-maps.yandex.ru/1.x/?apikey={yandex_token}&format=json&geocode={preprocLine}&lang=ru_RU'
    try:
        r = requests.get(request).json()
        coordinates = r['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split(' ')
        lat, lon = coordinates[1], coordinates[0]
    except Exception as error:
        print(error)
        return 0,0
    return float(lat),float(lon)

def searchHaversineDistance(lat1,lon1,lat2,lon2):
    cl1, cl2 = math.cos(lat1), math.cos(lat2)
    sl1, sl2 = math.sin(lat1), math.sin(lat2)
    delta = lon2 - lon1
    cdelta, sdelta = math.cos(delta), math.sin(delta)
    y = math.sqrt(math.pow(cl2*delta, 2) + math.pow(cl1*sl2-sl1*cl2*cdelta,2))
    x = sl1*sl2 + cl1*cl2*cdelta
    ad = math.atan2(y,x)
    return ad*rad

In [ ]:
vsp_df = pd.read_csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1rkxnMp0DI6JwhYea8-ykfA1G75zWQKyB').rename(columns={"адрес":"Адрес"})
res = [searchCoordinates(x) for x in tqdm(vsp_df['Адрес'].tolist()) ]
vsp_df['Широта'], vsp_df['Долгота'] = zip(*res)

In [ ]:
vsp_df[vsp_df['Широта']==0]

In [ ]:
pos_df2 = pd.read_csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1-CF7VKztxOoxvbtaCumA7mcDxxn640fA').rename(columns={"Адрес (сцепили)": "Адрес"})
pos_df = pos_df2[['Адрес','Широта','Долгота']]
pos_df = pos_df.drop_duplicates(subset=['Адрес'])

In [ ]:
vsp_df['ВСП/ПОСТАМАТ'] = ['ВСП' for _ in range(len(vsp_df))]
pos_df['ВСП/ПОСТАМАТ'] = ['ПОСТАМАТ' for _ in range(len(pos_df))]
res_df = pd.concat([vsp_df, pos_df])

## ПОСЛЕ ПРЕОБРАЗОВАНИЯ ФАЙЛОВ ИСПОЛЬЗУЕТСЯ ПОЛНЫЙ СПИСОК АДРЕСОВ

In [2]:
res_df = pd.read_csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1pOO07xTqDrxayIjZ0lVXTl1UA37rj-CK')

In [3]:
coords = res_df[['Широта','Долгота']].values

In [4]:
kms_per_radian = 6371.0088
epsilon = 0.1 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
res_df['Класс'] = db.labels_cluster_labels = db.labels_

clusters = []
clusters_dict = dict(res_df['Класс'].value_counts())
for key in clusters_dict.keys():
    if clusters_dict[key]>1:
        clusters.append(key)
        
map_df = res_df[res_df['Класс'].isin(clusters)]

In [5]:
map_df.sample(10)

,Адрес,Широта,Долгота,ВСП/ПОСТАМАТ,Класс
5691,г Москва г Москва пр-кт Мира д. 180,55.824045,37.645693,ПОСТАМАТ,84
335,"г Москва, пр-кт Кутузовский, д 71 стр 1",55.723719,37.452103,ВСП,334
154,"г Москва, ул Ярцевская, д 25А",55.741019,37.415415,ВСП,154
503,"г. Санкт-Петербург, Бухарестская,43",59.860848,30.389377,ВСП,495
408,"г Москва, ул Череповецкая, д 20",55.897629,37.581047,ВСП,406
176,"г Москва, пр-кт Рязанский, д 69",55.718806,37.787147,ВСП,176
579,"г Санкт-Петербург, пр-кт Ленинский, д 71 к 1",59.859510,30.185046,ВСП,568
5035,г Москва г Москва ул Шолохова д. 16,55.640018,37.351084,ПОСТАМАТ,303
5204,г Санкт-Петербург г Санкт-Петербург пр-кт ...,59.835831,30.145125,ПОСТАМАТ,518
975,обл Свердловская г Нижний Тагил ул Металлу...,57.940072,59.997427,ПОСТАМАТ,961


In [6]:
colors_dict = {
    'ВСП': 'red',
    'ПОСТАМАТ': 'green'
}

x_centre, y_centre = 64.6863136, 97.7453061 # центр России
geo_map = folium.Map(location=[x_centre,y_centre],zoom_start=4,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(geo_map)

marker_cluster = MarkerCluster().add_to(geo_map)
for lat,lon, address, obj_type in map_df[['Широта','Долгота','Адрес','ВСП/ПОСТАМАТ']].values:
    folium.CircleMarker(location=[lat,lon], radius = 9, popup = str(address), fill_color = colors_dict[obj_type], color='gray', fill_opacity=0.9  ).add_to(marker_cluster)

In [7]:
geo_map

In [ ]:
df = pd.read_csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1-CF7VKztxOoxvbtaCumA7mcDxxn640fA')
df = df.drop_duplicates(subset=['Адрес (сцепили)'])
df['Широта(рад)'] = df['Широта'].apply(lambda x: x * pi / 180.0)
df['Долгота(рад)'] = df['Долгота'].apply(lambda x: x * pi / 180.0)
df = df.sort_values(by=['Широта'])
objs = list(zip(
    df['Широта(рад)'].tolist(),
    df['Долгота(рад)'].tolist(),
    df['Адрес (сцепили)'].tolist(),
    df['Широта'].tolist(),
    df['Долгота'].tolist()
))

In [ ]:
df['Широта'].mean()

In [ ]:
res_list = []
for i,fp in tqdm(enumerate(objs)):
    lat1, lon1 = fp[0], fp[1]
    address1 = fp[2]
    for sp in objs[i+1:-1]:
        lat2, lon2 = sp[0],sp[1]
        address2 = sp[2]
        dist = searchHaversineDistance(lat1,lon1,lat2,lon2)
        if dist>0 and dist<=100:
            objects_dict = {'Адрес(1)': address1,
                            'Широта(1)': lat1,
                            'Долгота(1)': lon1,
                            'Адрес(2)': address2,
                            'Широта(2)': lat2,
                            'Долгота(2)': lon2,
                            'Расстояние': dist}
            res_list.append(objects_dict)

In [ ]:
df2 = pd.DataFrame(res_list)

In [ ]:
coords = df[['Широта','Долгота']].values
kms_per_radian = 6371.0088
epsilon = 0.25 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
df['Cluster'] = db.labels_cluster_labels = db.labels_

In [ ]:
clusters = []
clusters_dict = dict(df['Cluster'].value_counts())
for key in clusters_dict.keys():
    if clusters_dict[key]>1:
        clusters.append(key)

In [ ]:
map_df = df[df['Cluster'].isin(clusters)]

In [ ]:
x_centre, y_centre = 64.6863136, 97.7453061
geo_map = folium.Map(location=[x_centre,y_centre],zoom_start=12,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(geo_map)

marker_cluster = MarkerCluster().add_to(geo_map)
for lat,lon, address in map_df[['Широта','Долгота','Адрес (сцепили)']].values:
    folium.CircleMarker(location=[lat,lon], radius = 9, popup = str(address), fill_color = 'red', color='gray', fill_opacity=0.9  ).add_to(marker_cluster)

In [ ]:
geo_map

In [ ]:
geo_map.save('res.html')

In [1]:
!pip install geocoder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 365.2 kB/s eta 0:00:00a 0:00:01


In [1]:
import warnings
warnings.filterwarnings("ignore")
import re
import pandas as pd
from collections import Counter
import json
import urllib
import requests
from tqdm import tqdm
import geocoder
import math
from tqdm import tqdm

ModuleNotFoundError: No module named 'geocoder'

In [ ]:
pickpoint = pd.read_csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1Z2T5Sv_G6dt_ZYSc7trF8b0Wle0LTh5H', sep='|')
vsps = pd.read_csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1KwPom658pEskjqEP7iZKDvIqG13nDOsZ', sep='|')
postamats = pd.read_csv('https://drive.google.com/uc?export=download&confirm=no_antivirus&id=12gfifNGd8Yzewo3ftAS1hqsMDgsckXAY',sep='|')

In [4]:
pick = pickpoint[['Адрес','Время работы','Широта','Долгота','Тип Пункта']]
pos = postamats[['Адрес постамата','Режим работы','Широта','Долгота']]
vsp = vsps[['Адрес','Режим работы ВСП/ПВЗ','Широта','Долгота']]
table_columns = ['Адрес','Режим работы','Широта','Долгота','Тип Пункта']
pos['Тип Пункта'] = ['ПОСТАМАТ СБЕРЛОГИСТИКИ' for _ in range(0,len(pos))]
pos.columns = table_columns
vsp['Тип Пункта'] = ['ВСП/ПВЗ СБЕРЛОГИСТИКИ' for _ in range(0, len(vsp))]
vsp.columns = table_columns
pick['Тип Пункта'] = pick['Тип Пункта'].apply(lambda x: f'{x} Pickpoint')
pick.columns = table_columns
con_df = pd.concat([pos,vsp,pick])

In [16]:
vsp

,Адрес,Режим работы,Широта,Долгота,Тип Пункта
0,"г Белгород,улица Князя Трубецкого, дом 24","ПН-ПТ 09:00-17:30, СБ 09:00-13:30, ВС выходной",50.59265,36.59621,ВСП/ПВЗ СБЕРЛОГИСТИКИ
1,"г Белгород,улица Некрасова, дом 5Б/8","ПН-ПТ 09:00-18:00, СБ 10:00-13:30, ВС выходной",50.61572,36.58259,ВСП/ПВЗ СБЕРЛОГИСТИКИ
2,"Белгородская область,город Алексеевка, пер 1-й...","ПН-ПТ 08:00-17:00, СБ 08:00-13:30, ВС выходной",50.62789,38.68997,ВСП/ПВЗ СБЕРЛОГИСТИКИ
3,"г Белгород,улица Железнякова, дом 18А","ПН-ПТ 09:00-18:00, СБ 09:00-13:00, ВС выходной",50.63256,36.57629,ВСП/ПВЗ СБЕРЛОГИСТИКИ
4,"Белгородская область,город Бирюч,улица Карла М...","ПН-ПТ 08:00-16:30, СБ 08:30-13:30, ВС выходной",50.65026,38.40225,ВСП/ПВЗ СБЕРЛОГИСТИКИ
...,...,...,...,...,...
5349,"Хабаровский край,город Бикин,улица Гагарина, д...","ПН-ПТ 08:30-18:00, СБ 09:00-14:00, ВС выходной",46.80912,134.25802,ВСП/ПВЗ СБЕРЛОГИСТИКИ
5350,"Хабаровский край,рабочий поселок Переяславка,у...","ПН-ПТ 08:30-17:30, СБ-ВС выходной",47.96517,135.05609,ВСП/ПВЗ СБЕРЛОГИСТИКИ
5351,"г Хабаровск,улица Автобусная, дом 2","ПН-ПТ 08:30-18:30, СБ 09:00-14:00, ВС выходной",48.36708,135.04322,ВСП/ПВЗ СБЕРЛОГИСТИКИ
5352,"Хабаровский край,рабочий поселок Ванино, бульв...","ПН-ПТ 09:00-18:30, СБ-ВС выходной",49.09064,140.26122,ВСП/ПВЗ СБЕРЛОГИСТИКИ


In [5]:
rad = 6372795

def searchHaversineDistance(lat1,lon1,lat2,lon2):
    lat1,lat2 = lat1*math.pi/180, lat2*math.pi/180
    lon1,lon2 = lon1*math.pi/180, lon2*math.pi/180

    cl1, cl2 = math.cos(lat1), math.cos(lat2)
    sl1, sl2 = math.sin(lat1), math.sin(lat2)
    delta = lon2 - lon1
    cdelta, sdelta = math.cos(delta), math.sin(delta)
    y = math.sqrt(math.pow(cl2*delta, 2) + math.pow(cl1*sl2-sl1*cl2*cdelta,2))
    x = sl1*sl2 + cl1*cl2*cdelta
    ad = math.atan2(y,x)
    return ad*rad

In [ ]:
x_centre, y_centre = 64.6863136, 97.7453061
geo_map = folium.Map(location=[x_centre,y_centre],zoom_start=12,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(geo_map)

marker_cluster = MarkerCluster().add_to(geo_map)

for 


for lat,lon, address in map_df[['Широта','Долгота','Адрес (сцепили)']].values:
    folium.CircleMarker(location=[lat,lon], radius = 9, popup = str(address), fill_color = 'red', color='gray', fill_opacity=0.9  ).add_to(marker_cluster)

In [6]:
result = []
all_data = con_df.values
for i,data in tqdm(enumerate(all_data)):
    address, work, lat, lon, obj_type = data[0], data[1], data[2], data[3], data[4]
    try:
        sliced_data = all_data[i+1:]
        for data2 in sliced_data:
            address2, work2, lat2,lon2, obj_type2 = data2[0], data2[1], data2[2], data2[3], data2[4]
            distance = searchHaversineDistance(lat,lon, lat2,lon2)
            if distance<=500:
                result.append([address, work, lat, lon, obj_type, address2, work2, lat2, lon2, obj_type2, distance])
    except Exception as error:
        print(error)

25674it [21:07, 20.25it/s] 


In [8]:
res_df = pd.DataFrame(result, columns=['Адрес1','Режим работы1','Широта1','Долгота1','Тип Пункта1','Адрес2','Режим работы2','Широта2','Долгота2','Тип Пункта2','Расстояние'])

In [12]:
res_df.sample(10)

,Адрес1,Режим работы1,Широта1,Долгота1,Тип Пункта1,Адрес2,Режим работы2,Широта2,Долгота2,Тип Пункта2,Расстояние
9950,"Свердловская область,город Нижняя Салда,улица ...",ПН-ВС 08:00-22:00,58.073300,60.694820,ПОСТАМАТ СБЕРЛОГИСТИКИ,"Свердловская область,город Нижняя Салда,улица ...","ПН-ПТ 09:00-18:00, СБ-ВС выходной",58.073980,60.692160,ВСП/ПВЗ СБЕРЛОГИСТИКИ,173.781296
12924,"Республика Татарстан,город Казань,улица Рашида...",ПН-ВС 09:00-21:00,55.783660,49.227890,ПОСТАМАТ СБЕРЛОГИСТИКИ,"Республика Татарстан,город Казань,улица Академ...","ПН-ПТ 08:30-19:30, СБ 08:30-15:00, ВС выходной",55.785140,49.232300,ПОСТАМАТ СБЕРЛОГИСТИКИ,321.207919
24815,"город Москва,Московская обл., Русаковская ул.,...",Ежедневно: 8-23,55.788201,37.681649,Постамат Pickpoint,"город Москва,Московская обл., Сокольническая п...",Ежедневно: 7-23,55.789906,37.678177,Постамат Pickpoint,288.282652
13928,"Республика Карелия,город Петрозаводск, район П...","ПН-ПТ 09:30-18:30, СБ 10:00-14:30, ВС выходной",61.798530,34.330670,ПОСТАМАТ СБЕРЛОГИСТИКИ,"г Петрозаводск, район Первомайский, проспект П...","ПН-ПТ 09:30-18:30, СБ 10:00-14:30, ВС выходной",61.798530,34.330670,ВСП/ПВЗ СБЕРЛОГИСТИКИ,0.000000
16736,"Орловская область,город Орёл,улица Московская,...","ПН-ПТ 00:00-23:59, СБ 00:00-23:59, ВС 00:00-23:59",52.968680,36.079600,ПОСТАМАТ СБЕРЛОГИСТИКИ,"город Орел,Орловская обл., Новосильская ул., д...",Ежедневно: 8-22:30,52.966404,36.082116,Постамат Pickpoint,304.123834
20188,"г Тверь,улица Трехсвятская, дом 8","ПН-ПТ 09:00-18:30, СБ 09:00-15:00, ВС 09:00-14:30",56.853160,35.909310,ВСП/ПВЗ СБЕРЛОГИСТИКИ,"город Тверь,Тверская обл., Тверской пр-кт, д. 2",Ежедневно: 8-23,56.853099,35.907684,Постамат Pickpoint,99.123450
15409,"Краснодарский край,город Краснодар,улица Герое...",ПН-ВС 09:00-19:00,45.065170,39.033580,ПОСТАМАТ СБЕРЛОГИСТИКИ,"город Краснодар,Краснодарский край, Черкасская...",Ежедневно: 10-20 (обед: 14-15),45.066713,39.038144,ПВЗ Pickpoint,397.496183
29708,"город Краснодар,Краснодарский край, 40-летия П...",Ежедневно: 8-22,45.056057,39.018214,Постамат Pickpoint,"город Краснодар,Краснодарский край, ул им. 40-...",Ежедневно: 9-21,45.055365,39.019735,Постамат Pickpoint,142.149376
16103,"Свердловская область,город Екатеринбург, Желез...","ПН-ПТ 00:00-23:59, СБ 00:00-23:59, ВС 00:00-23:59",0.000000,0.000000,ПОСТАМАТ СБЕРЛОГИСТИКИ,"Московская область,город Павловский Посад,улиц...",ПН-ВС 09:00-22:00,0.000000,0.000000,ПОСТАМАТ СБЕРЛОГИСТИКИ,0.000000
28706,"город Пермь,Пермский край, Луначарского ул., д...",Ежедневно: 7-23,58.010340,56.252665,Постамат Pickpoint,"город Пермь,Пермский край, Николая Островского...",Ежедневно: 7-23,58.011956,56.259309,Постамат Pickpoint,430.772470


In [9]:
res_df.to_csv('result_distance_frame.csv',sep='|')

In [10]:
map_df = res_df[res_df['Расстояние']<=300]

In [11]:
map_df.shape

(17611, 11)

In [17]:
zero = map_df[map_df['Расстояние']==0]

In [19]:
zero.shape

(5048, 11)

In [20]:
zero

,Адрес1,Режим работы1,Широта1,Долгота1,Тип Пункта1,Адрес2,Режим работы2,Широта2,Долгота2,Тип Пункта2,Расстояние
1,"Воронежская область,город Воронеж, бульвар Поб...","ПН-ПТ 00:00-23:59, СБ 00:00-23:59, ВС 00:00-23:59",51.713370,39.167080,ПОСТАМАТ СБЕРЛОГИСТИКИ,"г Воронеж, бульвар Победы, дом 22А","ПН-ПТ 09:00-18:00, СБ-ВС выходной",51.713370,39.167080,ВСП/ПВЗ СБЕРЛОГИСТИКИ,0.0
3,"Приморский край,город Владивосток, проспект Кр...","ПН-ПТ 09:00-19:00, СБ 10:00-16:30, ВС выходной",43.127310,131.907080,ПОСТАМАТ СБЕРЛОГИСТИКИ,"г Владивосток, проспект Красного Знамени, дом 59","ПН-ПТ 09:00-19:00, СБ 10:00-16:30, ВС выходной",43.127310,131.907080,ВСП/ПВЗ СБЕРЛОГИСТИКИ,0.0
6,"г Москва, Старопетровский проезд, дом 1 строен...","ПН-ПТ 00:00-23:59, СБ 00:00-23:59, ВС 00:00-23:59",55.823160,37.500810,ПОСТАМАТ СБЕРЛОГИСТИКИ,"г Москва, проезд Старопетровский, дом 1 строен...","ПН-ПТ 09:30-20:00, СБ 10:00-16:00, ВС выходной",55.823160,37.500810,ВСП/ПВЗ СБЕРЛОГИСТИКИ,0.0
9,"г Москва, Тимирязевский район , Дмитровское шо...","ПН-ПТ 00:00-23:59, СБ 00:00-23:59, ВС 00:00-23:59",55.848410,37.566280,ПОСТАМАТ СБЕРЛОГИСТИКИ,"г Москва, ш Дмитровское, дом 65 корпус 1","ПН-ПТ 09:00-20:00, СБ-ВС выходной",55.848410,37.566280,ВСП/ПВЗ СБЕРЛОГИСТИКИ,0.0
13,"г Москва, Коровинское шоссе, дом 20 корпус 1","ПН-ПТ 08:30-19:30, СБ 09:00-15:30, ВС выходной",55.879624,37.521673,ПОСТАМАТ СБЕРЛОГИСТИКИ,"г Москва, ш Коровинское, дом 20 корпус 1","ПН-ПТ 08:30-19:30, СБ 09:00-15:30, ВС выходной",55.879624,37.521673,ВСП/ПВЗ СБЕРЛОГИСТИКИ,0.0
...,...,...,...,...,...,...,...,...,...,...,...
30184,"город Электросталь,Московская обл., Ленина пр-...",Ежедневно: 10-21,55.790570,38.441013,ПВЗ Pickpoint,"город Электросталь,Московская обл., Ленина пр-...",Пн-Пт: 10-19; Сб: 10-17; Вс: Выходной,55.790570,38.441013,Постамат Pickpoint,0.0
30328,"город Малаховка,Московская обл., Быковское шос...",Ежедневно: 10-22,55.625293,38.011879,ПВЗ Pickpoint,"город Малаховка,Московская обл., Быковское шос...",Ежедневно: 10-22,55.625293,38.011879,Постамат Pickpoint,0.0
30385,"город Волхов,Ленинградская обл., Советская ул....",Пн: Выходной; Вт-Сб: 10-19 (обед: 14-15); Вс: ...,59.928095,32.304002,ПВЗ Pickpoint,"город Волхов,Ленинградская обл., Советская ул....",Ежедневно: 9-21,59.928095,32.304002,Постамат Pickpoint,0.0
30864,"город Санкт-Петербург,Ленинградская обл., Олек...",Ежедневно: 10-21,59.833190,30.421366,Постамат Pickpoint,"город Санкт-Петербург,Ленинградская обл., Олек...",Ежедневно: 10-21,59.833190,30.421366,ПВЗ Pickpoint,0.0
